In [1]:
# Importar las librerías necesarias
from flask import Flask, request, jsonify
import requests
import os
import logging
from dotenv import load_dotenv
import re

# Cargar variables de entorno para mayor seguridad
load_dotenv()
API_KEY = os.getenv("NCBI_API_KEY")

# Crear la instancia de la app Flask
app = Flask(__name__)

# Configuración de seguridad
app.config["JSONIFY_PRETTYPRINT_REGULAR"] = False
app.secret_key = os.urandom(24)  # Clave secreta para proteger cookies y sesiones

# Configurar logging seguro
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger()

# Endpoints base de NCBI
NCBI_BASE_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
NCBI_FETCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

# Función para validar entradas (sanitización)
def sanitize_input(user_input):
    return re.sub(r'[^a-zA-Z0-9_ ]', '', user_input)

# Ruta principal para buscar datos RNA-seq de hipocampo del cerebro
@app.route('/fetch_rnaseq_data', methods=['POST'])
def fetch_rnaseq_data():
    try:
        # Obtener los parámetros de búsqueda desde la solicitud
        data = request.get_json()
        organism = sanitize_input(data.get('organism', ''))
        tissue = sanitize_input(data.get('tissue', 'hippocampus'))
        data_type = sanitize_input(data.get('data_type', 'rna-seq'))

        if not organism:
            return jsonify({"error": "Organism field is required."}), 400

        # Construir la consulta segura para NCBI
        query = f"{organism} {tissue} {data_type}"
        params = {
            'db': 'gds',
            'term': query,
            'retmode': 'json',
            'api_key': API_KEY
        }

        # Hacer la solicitud al endpoint de búsqueda de NCBI
        response = requests.get(NCBI_BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        result = response.json()

        # Procesar los IDs de los estudios encontrados
        id_list = result.get('esearchresult', {}).get('idlist', [])
        if not id_list:
            return jsonify({"message": "No data found for the specified query."}), 404

        # Obtener detalles de cada estudio encontrado
        details = []
        for study_id in id_list:
            fetch_params = {
                'db': 'gds',
                'id': study_id,
                'retmode': 'xml',
                'api_key': API_KEY
            }
            fetch_response = requests.get(NCBI_FETCH_URL, params=fetch_params, timeout=10)
            fetch_response.raise_for_status()
            details.append(fetch_response.text)

        # Guardar los datos en el sistema local para su posterior análisis
        with open('ncbi_rnaseq_data.xml', 'w') as file:
            file.writelines(details)

        return jsonify({"message": "Data fetched and saved successfully.", "study_ids": id_list}), 200

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching data from NCBI: {e}")
        return jsonify({"error": "Failed to fetch data from NCBI."}), 500
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        return jsonify({"error": "An unexpected error occurred."}), 500

# Medida de seguridad: limitar métodos HTTP
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "API is running."}), 200

# Ejecutar la aplicación de Flask
if __name__ == '__main__':
    # Solo permitir conexiones locales (evitar exposición pública durante desarrollo)
    app.run(host='127.0.0.1', port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


2024-12-04 00:40:45,268 INFO WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2024-12-04 00:40:45,269 INFO Press CTRL+C to quit
2024-12-04 00:42:50,361 INFO 127.0.0.1 - - [04/Dec/2024 00:42:50] "GET /health HTTP/1.1" 200 -
2024-12-04 00:45:17,097 INFO 127.0.0.1 - - [04/Dec/2024 00:45:17] "POST /fetch_rnaseq_data HTTP/1.1" 200 -
